I am sincerely grateful to the classmate whose code provided invaluable reference. If my publicly shared notebook inadvertently causes any issues, I am DEEPLY SORRY for any inconvenience caused. 😔

I became aware of this competition and its associated code through a social chat group.

With the original author's kind permission, I have referenced code from KoHaKu-Lab(check in leaderboard)(Course Information: TAICA AIA5320).
Code from https://github.com/KohakuBlueleaf/KohakuRAG

In [ ]:
!nvidia-smi
!nvcc --version
!pip install openai rank_bm25 sentence-transformers transformers[torch] faiss-cpu

In [ ]:
import os

base = "/kaggle/input"
dirs = os.listdir(base)
dirs

import pandas as pd

base = "/kaggle/input/WattBot2025"


def smart_read_csv(path):
    encodings = ["utf-8", "latin1", "ISO-8859-1", "cp1252"]
    for enc in encodings:
        try:
            return pd.read_csv(path, encoding=enc)
        except Exception as e:
            last_error = e
    raise last_error


train_df = pd.read_csv(f"{base}/train_QA.csv")
test_df = pd.read_csv(f"{base}/test_Q.csv")
meta_df = smart_read_csv(f"{base}/metadata.csv")

train_df.head(), test_df.head(), meta_df.head()

In [ ]:
def load_metadata(path: Path) -> list[dict[str, str]]:
    with path.open(newline="", encoding="utf-8", errors="replace") as f:
        return list(csv.DictReader(f))


def clean_url(url: str) -> str:
    return (url or "").strip()


def is_pdf_url(url: str) -> bool:
    cleaned = clean_url(url)
    if not cleaned:
        return False
    parts = urlsplit(cleaned)
    base = f"{parts.scheme}://{parts.netloc}{parts.path}".rstrip("/")
    path_lower = parts.path.lower().rstrip("/")
    if base.lower().endswith(".pdf"):
        return True
    # arXiv-style URLs sometimes omit the .pdf suffix but still live under /pdf/.
    if parts.netloc.endswith("arxiv.org") and path_lower.startswith("/pdf/"):
        return True
    return False


def has_pdf_mime(url: str) -> bool:
    try:
        resp = requests.head(url, allow_redirects=True, timeout=30)
        ctype = resp.headers.get("Content-Type", "").lower()
        return "pdf" in ctype
    except requests.RequestException:
        return False


def download_pdf(url: str, dest: Path, *, force: bool = False) -> None:
    if dest.exists() and not force:
        return
    dest.parent.mkdir(parents=True, exist_ok=True)
    headers = {"User-Agent": "KohakuRAG/0.0.1"}
    resp = requests.get(url, headers=headers, timeout=120)
    resp.raise_for_status()
    dest.write_bytes(resp.content)


parser = argparse.ArgumentParser(
    description="Fetch WattBot source PDFs and convert to structured JSON."
)
parser.add_argument("--metadata", type=Path, default=Path(f"{base}/metadata.csv"))
parser.add_argument("--pdf-dir", type=Path, default=Path("artifacts/raw_pdfs"))
parser.add_argument("--output-dir", type=Path, default=Path("artifacts/docs"))
parser.add_argument("--force-download", action="store_true")
parser.add_argument(
    "--limit", type=int, default=5, help="Process only N documents (for testing)."
)

args = parser.parse_args()
# Load document metadata
rows = load_metadata(args.metadata)
processed = 0
skipped = 0

# Process each document
for row in rows:
    doc_id = row["id"]
    title = row.get("title") or doc_id
    url = clean_url(row.get("url") or "")

    # Validate URL
    if not url:
        skipped += 1
        print(f"[skip] {doc_id}: missing URL", file=sys.stderr)
        continue

    if not is_pdf_url(url) and not has_pdf_mime(url):
        skipped += 1
        print(f"[skip] {doc_id}: URL does not look like PDF ({url})", file=sys.stderr)
        continue

    pdf_path = args.pdf_dir / f"{doc_id}.pdf"
    json_path = args.output_dir / f"{doc_id}.json"

    try:
        # Download PDF
        download_pdf(url, pdf_path, force=args.force_download)

        # Parse PDF into structured payload
        payload = pdf_to_document_payload(
            pdf_path,
            doc_id=doc_id,
            title=title,
            metadata={
                "url": url,
                "type": row.get("type"),
                "year": row.get("year"),
            },
        )

        # Save as JSON
        json_path.parent.mkdir(parents=True, exist_ok=True)
        json_path.write_text(
            json.dumps(payload_to_dict(payload), ensure_ascii=False),
            encoding="utf-8",
        )

        processed += 1
        print(f"[ok] {doc_id} -> {json_path}")

    except requests.HTTPError as err:
        skipped += 1
        print(f"[error] {doc_id}: download failed ({err})", file=sys.stderr)
    except Exception as exc:
        skipped += 1
        print(f"[error] {doc_id}: conversion failed ({exc})", file=sys.stderr)

    # Respect limit for testing
    if args.limit and processed >= args.limit:
        break

print(
    f"Processed {processed} documents, skipped {skipped}. "
    f"Structured docs saved under {args.output_dir}"
)

In [ ]:
from pathlib import Path
from typing import Any

from pypdf import PdfReader
from pypdf.generic import DictionaryObject, IndirectObject

from .text_utils import split_paragraphs, split_sentences
from .types import (
    DocumentPayload,
    ParagraphPayload,
    SectionPayload,
    SentencePayload,
)


def _resolve(obj):
    return obj.get_object() if isinstance(obj, IndirectObject) else obj


def _extract_images(page) -> list[dict[str, Any]]:
    images: list[dict[str, Any]] = []
    resources = page.get("/Resources")
    if not resources:
        return images
    resources = _resolve(resources)
    xobject = (
        resources.get("/XObject") if isinstance(resources, DictionaryObject) else None
    )
    if xobject is None:
        return images
    xobject = _resolve(xobject)
    if not isinstance(xobject, DictionaryObject):
        return images
    for name, obj in xobject.items():
        resolved = _resolve(obj)
        if not isinstance(resolved, DictionaryObject):
            continue
        subtype = resolved.get("/Subtype")
        if subtype == "/Image":
            images.append(
                {
                    "name": str(name),
                    "width": resolved.get("/Width"),
                    "height": resolved.get("/Height"),
                    "color_space": resolved.get("/ColorSpace"),
                }
            )
    return images


def pdf_to_document_payload(
    pdf_path: Path,
    *,
    doc_id: str,
    title: str,
    metadata: dict[str, Any],
) -> DocumentPayload:
    reader = PdfReader(str(pdf_path))
    sections: list[SectionPayload] = []
    all_paragraph_texts: list[str] = []
    for page_num, page in enumerate(reader.pages, start=1):
        raw_text = page.extract_text() or ""
        paragraphs = []
        for paragraph_text in split_paragraphs(raw_text):
            sentences = [
                SentencePayload(text=sentence)
                for sentence in split_sentences(paragraph_text)
            ]
            paragraphs.append(
                ParagraphPayload(
                    text=paragraph_text,
                    sentences=sentences or None,
                    metadata={"page": page_num},
                )
            )
            all_paragraph_texts.append(paragraph_text)
        images = _extract_images(page)
        for idx, info in enumerate(images, start=1):
            caption = (
                f"[Image page={page_num} idx={idx}] Placeholder for "
                f"{info.get('width')}x{info.get('height')} {info.get('color_space')} graphic."
            )
            sentences = [SentencePayload(text=caption)]
            paragraphs.append(
                ParagraphPayload(
                    text=caption,
                    sentences=sentences,
                    metadata={
                        "page": page_num,
                        "image_index": idx,
                        "placeholder": True,
                    },
                )
            )
            all_paragraph_texts.append(caption)
        if paragraphs:
            sections.append(
                SectionPayload(
                    title=f"Page {page_num}",
                    paragraphs=paragraphs,
                    metadata={"page": page_num},
                )
            )
    combined_text = "\n\n".join(all_paragraph_texts)
    return DocumentPayload(
        document_id=doc_id,
        title=title,
        text=combined_text,
        metadata=metadata,
        sections=sections,
    )


def pdf_to_markdown(
    pdf_path: Path,
    *,
    doc_id: str,
    title: str,
    metadata: dict[str, Any],
) -> str:
    payload = pdf_to_document_payload(
        pdf_path, doc_id=doc_id, title=title, metadata=metadata
    )
    lines = [f"# {title}", ""]
    for section in payload.sections or []:
        lines.append(f"## {section.title}")
        lines.append("")
        for paragraph in section.paragraphs:
            lines.append(paragraph.text)
            lines.append("")
    return "\n".join(lines)

In [ ]:
import json
import sqlite3
import hashlib
import faiss
import torch
import numpy as np
import pickle
from pathlib import Path
from typing import List, Dict, Any
from rank_bm25 import BM25Okapi
from transformers import AutoModel, AutoTokenizer
from transformers import AutoModelForSequenceClassification

# Embedding Model (chache)
os.environ["HF_HUB_ENABLE_HF_CONTRIBUTIONS"] = "1"


class EmbeddingModel:
    def __init__(self, model_name="jinaai/jina-embeddings-v3"):
        self.tokenizer = AutoTokenizer.from_pretrained(
            model_name, trust_remote_code=True
        )
        self.model = AutoModel.from_pretrained(model_name, trust_remote_code=True)

        # Optional: Move model to GPU if available for faster inference
        if torch.cuda.is_available():
            self.model.to("cuda")

        self.cache_db = Path("embed_cache.db")
        self._init_cache()

    def _init_cache(self):
        conn = sqlite3.connect(self.cache_db)
        conn.execute(
            """
            CREATE TABLE IF NOT EXISTS cache (
                hash TEXT PRIMARY KEY,
                embedding BLOB
            )
        """
        )
        conn.commit()
        conn.close()

    def _hash(self, text: str):
        return hashlib.md5(text.encode()).hexdigest()

    def embed(self, texts: List[str]):
        out = []
        conn = sqlite3.connect(self.cache_db)
        cur = conn.cursor()

        for t in texts:
            key = self._hash(t)
            row = cur.execute(
                "SELECT embedding FROM cache WHERE hash=?", (key,)
            ).fetchone()

            if row:  # HIT
                emb = pickle.loads(row[0])
                out.append(emb)
                continue

            # MISS → compute
            # Move input tensors to the same device as the model (CPU or GPU)
            tok = self.tokenizer(t, return_tensors="pt", truncation=True)
            if torch.cuda.is_available():
                tok = {k: v.to("cuda") for k, v in tok.items()}

            with torch.no_grad():
                h = self.model(**tok).last_hidden_state.mean(dim=1).squeeze()
                # Convert to float32 before calling .numpy()
                h = h.cpu().to(torch.float32).numpy()

            cur.execute(
                "INSERT OR REPLACE INTO cache VALUES (?,?)", (key, pickle.dumps(h))
            )
            out.append(h)

        conn.commit()
        conn.close()
        return out


class FaissCPUIndex:
    def __init__(self, dim=1024, nlist=1, m=8):
        """
        IVF-PQ：适用于大量文档（>1M），小数据集时自动调整 nlist
        dim: embedding 维度
        nlist: 聚类数量 (如果小于 1 会自动设为 1)
        m: PQ 分段数 (dim % m == 0)
        """
        self.dim = dim
        self.nlist = max(1, nlist)
        self.m = m

        quantizer = faiss.IndexFlatL2(dim)
        self.index = faiss.IndexIVFPQ(quantizer, dim, self.nlist, m, 8)

        self.is_trained = False
        self.doc_texts = []
        self.doc_ids = []

    def train(self, embeddings: np.ndarray):
        if not self.is_trained:
            # Faiss IVF 训练要求 num_embeddings >= nlist
            nlist = min(self.nlist, embeddings.shape[0])
            if nlist < self.nlist:
                self.index.nlist = nlist
            self.index.train(embeddings)
            self.is_trained = True

    def add(self, embeddings: np.ndarray, doc_ids, raw_texts):
        self.train(embeddings)
        self.index.add(embeddings)
        self.doc_ids.extend(doc_ids)
        self.doc_texts.extend(raw_texts)

    def search(self, query_emb, topk):
        scores, I = self.index.search(query_emb, topk)
        return [
            (self.doc_ids[i], self.doc_texts[i], float(scores[0][j]))
            for j, i in enumerate(I[0])
            if i >= 0
        ]


# BM25 Retriever
class BM25Retriever:
    def __init__(self, docs: List[str]):
        self.docs = docs
        self.tok = [d.split() for d in docs]
        self.bm25 = BM25Okapi(self.tok)

    def search(self, query: str, topk: int):
        scores = self.bm25.get_scores(query.split())
        idx = scores.argsort()[-topk:][::-1]
        return [(i, self.docs[i], float(scores[i])) for i in idx]


# Reranker (BGE)
class Reranker:
    def __init__(self, model_name="BAAI/bge-reranker-large"):
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModelForSequenceClassification.from_pretrained(model_name)

    def rerank(self, query: str, candidates: List[Dict[str, Any]], topk: int):
        pairs = [(query, c["text"]) for c in candidates]
        tok = self.tokenizer(
            [p[0] for p in pairs],
            [p[1] for p in pairs],
            return_tensors="pt",
            truncation=True,
            padding=True,
        )
        with torch.no_grad():
            scores = self.model(**tok).logits.squeeze().numpy()

        ranked = sorted(
            [
                {"id": c["id"], "text": c["text"], "score": float(s)}
                for c, s in zip(candidates, scores)
            ],
            key=lambda x: -x["score"],
        )
        return ranked[:topk]


# LLM Wrapper (OpenAI Compatible)
from openai import OpenAI


class ChatModel:
    def __init__(self, model="gpt-4o-mini", system_prompt=""):
        self.client = OpenAI()
        self.model = model
        self.system_prompt = system_prompt

    def complete(self, prompt):
        msg = self.client.chat.completions.create(
            model=self.model,
            messages=[
                {"role": "system", "content": self.system_prompt},
                {"role": "user", "content": prompt},
            ],
        )
        return msg.choices[0].message.content


# Main RAG Pipeline
class RAGPipeline:
    def __init__(self, store, embedder, chat_model, bm25=None, reranker=None):
        self.store = store  # FaissGPUIndex
        self.embedder = embedder  # EmbeddingModel
        self.chat = chat_model  # ChatModel
        self.bm25 = bm25  # BM25Retriever
        self.reranker = reranker  # Reranker

    def retrieve(self, question, topk=10):
        emb = self.embedder.embed([question])[0]
        emb = emb.reshape(1, -1).astype("float32")
        # 1) Faiss ANN
        ann_hits = self.store.search(emb, topk)
        # 2) BM25
        bm25_hits = []
        if self.bm25:
            bm25_hits = self.bm25.search(question, topk)
        # 合并 + 去重
        merged = {}
        for did, text, score in ann_hits + bm25_hits:
            merged[did] = {"id": did, "text": text, "score": score}
        merged_list = list(merged.values())

        # 3) Reranker
        if self.reranker:
            merged_list = self.reranker.rerank(question, merged_list, topk)

        return merged_list

    def run_qa(self, question, system_prompt, user_template, additional_info, top_k=5):
        ctx_docs = self.retrieve(question, top_k)

        context = "\n\n".join([f"[{c['id']}]\n{c['text']}" for c in ctx_docs])

        user_prompt = user_template.format(
            question=question,
            context=context,
            additional_info_json=json.dumps(additional_info),
        )

        raw = self.chat.complete(user_prompt)

        try:
            structured = json.loads(raw[raw.index("{") : raw.rindex("}") + 1])
        except:
            structured = {
                "answer": "is_blank",
                "answer_value": "is_blank",
                "ref_id": "is_blank",
                "explanation": "is_blank",
            }

        # 模拟 KohakuRAG 的标准输出格式
        return type(
            "QAResult",
            (),
            {"answer": structured, "raw_response": raw, "prompt": user_prompt},
        )


# pipeline


def build_pipeline(docs: List[str], ids: List[str]):
    embedder = EmbeddingModel()
    embs = embedder.embed(docs)
    embs = [e.astype("float32") for e in embs]

    embs_np = torch.tensor(embs).numpy()
    print(embs_np.shape[1])
    faiss_index = FaissCPUIndex(dim=embs_np.shape[1])
    faiss_index.add(embs_np, ids, docs)

    bm25 = BM25Retriever(docs)
    reranker = Reranker()

    chat = ChatModel(model="gpt-4o-mini")

    pipeline = RAGPipeline(
        store=faiss_index,
        embedder=embedder,
        chat_model=chat,
        bm25=bm25,
        reranker=reranker,
    )
    return pipeline

In [ ]:
import faiss

print("FAISS version:", faiss.__version__)
print("Has GPU module:", hasattr(faiss, "GpuIndexFlatL2"))
print("Available attributes:", [a for a in dir(faiss) if "Gpu" in a])

In [ ]:
# buld up these parts and test
docs = [
    "Solar panel efficiency depends on irradiance.",
    "Wind turbines convert kinetic energy to electricity.",
]
ids = ["doc1", "doc2"]

pipeline = build_pipeline(docs, ids)
qa = pipeline.run_qa(
    question="What affects solar panel efficiency?",
    system_prompt="You are helpful.",
    user_template="Q: {question}\nContext:\n{context}\n\nA:",
    additional_info={"answer_unit": ""},
    top_k=5,
)
print(qa.answer)